# Google Palm LLM in LangChain

In [78]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
from langchain.document_loaders.json_loader import JSONLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate


In [9]:
# load environment variables
load_dotenv()

True

In [11]:
# load GooglePalm
llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0)

In [12]:
# test llm
hello = llm("Hi. How are you?")
print(hello)

Doing okay.


## Load data from json file

In [56]:
loader = JSONLoader(file_path="../data/Ecommerce_FAQ_Chatbot_dataset.json",
                    jq_schema='.questions[].question',
                    text_content=True)

# Store the loaded data in the 'data' variable
data = loader.load()

In [57]:
data[0]

Document(page_content='How can I create an account?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 1})

## Hugging Face Embeddings

In [65]:
# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is the process for submitting a product review?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [67]:
len(e)

768

## Store vectors using FAISS

In [75]:
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [71]:
rdocs = retriever.get_relevant_documents("What is the process for submitting a product review?")
rdocs

[Document(page_content='How can I leave a product review?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 20}),
 Document(page_content='How can I contact customer support?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 10}),
 Document(page_content='Can I request a product demonstration before making a purchase?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 36}),
 Document(page_content='Can I request a product if it is not listed on your website?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 65})]

In [77]:
rdocs = retriever.get_relevant_documents("What is my name?")
rdocs

[Document(page_content='How can I create an account?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 1}),
 Document(page_content='Are my personal and payment details secure?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 14}),
 Document(page_content='How can I track my order?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 3}),
 Document(page_content='Do you have a loyalty program?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 16})]

## Create RetrievalQA chain along with prompt template

Somtimes LLMs might start making things up. For instance, when we ask them something not in the data, they might use what they know to guess an answer. The PromptTemplate helps us tell them what to do. For example, we can tell them to say "I don't know" if the answer isn't in the data.

In [79]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "answer" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


In [80]:
chain("What is my name?")

{'query': 'What is my name?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='How can I create an account?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 1}),
  Document(page_content='Are my personal and payment details secure?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 14}),
  Document(page_content='How can I track my order?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 3}),
  Document(page_content='Do you have a loyalty program?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 16})]}

In [82]:
chain("What is the process for submitting a product review?")

{'query': 'What is the process for submitting a product review?',
 'result': 'To leave a product review, please follow these steps:\n\n1. Navigate to the product page.\n2. Click on the "Write a Review" link.\n3. Enter your name and email address.\n4. Write your review.\n5. Click on the "Submit" button.',
 'source_documents': [Document(page_content='How can I leave a product review?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 20}),
  Document(page_content='How can I contact customer support?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Ecommerce_FAQ_Chatbot_dataset.json', 'seq_num': 10}),
  Document(page_content='Can I request a product demonstration before making a purchase?', metadata={'source': '/Users/armanbabayan/Personal Projects/Ecommerce-QA-Platform-with-Google-Palm-in-LangChain/data/Eco